# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']


Arxiv has 60 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2307.10277


extracting tarball to tmp_2307.10277...

 done.


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:473: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

✔ → 118975:\section{\IS diagrams for all PHANGS-MUSE galaxies}
  ↳ 134107:end


K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']


Found 155 bibliographic references in tmp_2307.10277/Ionized_gas_dispersion_in_PHANGS.bbl.
Retrieving document from  https://arxiv.org/e-print/2307.10839


extracting tarball to tmp_2307.10839... done.


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2307.10839/follow-up_photon_search.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'acknowledgments' from 'tmp_2307.10839/acknowledgments.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'aastex_authorlist' from 'tmp_2307.10839/aastex_authorlist.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.10277-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.10277) | **Quantifying the energy balance between the turbulent ionised gas and  young stars**  |
|| O. V. Egorov, et al. -- incl., <mark>K. Kreckel</mark>, <mark>E. Schinnerer</mark> |
|*Appeared on*| *2023-07-21*|
|*Comments*| *31 pages (including 5 pages in appendix), 19 figures, the abstract is abridged; submitted to A&A (in mid May; awaiting report)*|
|**Abstract**| We investigate the ionised gas morphology, excitation properties, and kinematics in 19 nearby star-forming galaxies from the PHANGS-MUSE survey. We directly compare the kinetic energy of expanding superbubbles and the turbulent motions in the interstellar medium with the mechanical energy deposited by massive stars in the form of winds and supernovae, with the aim to answer whether the stellar feedback is responsible for the observed turbulent motions and to quantify the fraction of mechanical energy retained in the superbubbles. Based on the distribution of the flux and velocity dispersion in the H$\alpha$ line, we select 1484 regions of locally elevated velocity dispersion ($\sigma$(H$\alpha$)>45 km/s), including at least 171 expanding superbubbles. We analyse these regions and relate their properties to those of the young stellar associations and star clusters identified in PHANGS-HST data. We find a good correlation between the kinetic energy of the ionised gas and the total mechanical energy input from supernovae and stellar winds from the stellar associations, with a typical efficiency of 10-20%. The contribution of mechanical energy by the supernovae alone is not sufficient to explain the measured kinetic energy of the ionised gas, which implies that pre-supernova feedback in the form of radiation/thermal pressure and winds is necessary. We find that the gas kinetic energy decreases with metallicity for our sample covering Z=0.5-1.0 Zsun, reflecting the lower impact of stellar feedback. For the sample of superbubbles, we find that about 40% of the young stellar associations are preferentially located in their rims. We also find a slightly higher (by ~15%) fraction of the youngest (1-2.5 Myr) stellar associations in the rims of the superbubbles than in the centres, and the opposite for older associations, which implies possible propagation or triggering of star formation. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.10839-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.10839) | **Search for UHE Photons from Gravitational Wave Sources with the Pierre  Auger Observatory**  |
|| P. A. Collaboration, et al. |
|*Appeared on*| *2023-07-21*|
|*Comments*| *15 pages*|
|**Abstract**| A search for time-directional coincidences of ultra-high-energy (UHE) photons above 10 EeV with gravitational wave (GW) events from the LIGO/Virgo runs O1 to O3 is conducted with the Pierre Auger Observatory. Due to the distinctive properties of photon interactions and to the background expected from hadronic showers, a subset of the most interesting GW events is selected based on their localization quality and distance. Time periods of 1000 s around and 1 day after the GW events are analyzed. No coincidences are observed. Upper limits on the UHE photon fluence from a GW event are derived that are typically at $\sim$7 MeV cm$^{-2}$ (time period 1000~s) and $\sim$35 MeV cm$^{-2}$ (time period 1 day). Due to the proximity of the binary neutron star merger GW170817, the energy of the source transferred into UHE photons above 40 EeV is constrained to be less than 20% of its total gravitational wave energy. These are the first limits on UHE photons from GW sources. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2307.10277.md
    + _build/html/tmp_2307.10277/./figs/NGC4254_isigma.png
    + _build/html/tmp_2307.10277/./figs/energy_ha_all_snw.png
    + _build/html/tmp_2307.10277/./figs/energy_ha_all_sn.png
    + _build/html/tmp_2307.10277/./figs/energy_ha_all_snw_s2dens.png
    + _build/html/tmp_2307.10277/./figs/energy_ha_all_sn_s2dens.png
    + _build/html/tmp_2307.10277/./figs/SB99_energy.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\SII}{[S~{\sc ii}]}$
$\newcommand{\OIII}{[O~{\sc iii}]}$
$\newcommand{\NII}{[N~{\sc ii}]}$
$\newcommand{\HII}{H~{\sc ii}}$
$\newcommand{\HeII}{He~{\sc ii}}$
$\newcommand{\HI}{H~{\sc i}}$
$\newcommand{\Ha}{H\alpha}$
$\newcommand{\Hb}{H\beta}$
$\newcommand{\kms}{ \mbox{km} \mbox{s}^{-1}}$
$\newcommand{\ergs}{ \mbox{erg} \mbox{s}^{-1}}$
$\newcommand{\HST}{\textit{HST}}$
$\newcommand{\SIIHa}{[S~{\sc ii}]/H\alpha}$
$\newcommand{\NIIHa}{[N~{\sc ii}]/H\alpha}$
$\newcommand{\OIIIHb}{[O~{\sc iii}]/H\beta}$
$\newcommand{\IS}{I - \sigma}$
$\newcommand{\SBDIG}{\mathrm{\Sigma(H\alpha)_{DIG}}}$
$\newcommand{\sigmaHa}{\sigma(H\alpha)}$
$\newcommand{\nregs}{1484}$
$\newcommand{\nregsnohst}{188}$
$\newcommand{\nsnr}{572}$
$\newcommand{\nregsnooldcls}{364}$
$\newcommand{\nregshaveyoungcls}{264}$
$\newcommand{\nshells}{171}$
$\newcommand{\nabmig}{741}$
$\newcommand{\ncentralpeak}{572}$
$\newcommand{\nregswithcls}{511}$
$\newcommand{\be}{\begin{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand{\red}{\textcolor{red}}$
$\newcommand{\green}{\textcolor{green}}$
$\begin{document}$
$   \title{Quantifying the energy balance between the turbulent ionised gas and young stars}$
$   \author{Oleg~V.~Egorov$
$          \inst{1}\fnmsep\thanks{\email{oleg.egorov@uni-heidelberg.de}}$
$          \and$
$          Kathryn Kreckel\inst{1}$
$          \and Simon~C.~O.~Glover\inst{2}$
$          \and Brent Groves\inst{3}$
$          \and$
$          Francesco Belfiore\inst{4}$
$          \and$
$          Eric Emsellem\inst{5,6}$
$          \and$
$          Ralf~S.~Klessen\inst{2,7}$
$          \and$
$          Adam K. Leroy\inst{8,9}$
$          \and$
$          Sharon E. Meidt\inst{10}$
$          \and$
$          Sumit K. Sarbadhicary\inst{8,9}$
$          \and$
$          Eva Schinnerer\inst{11}$
$          \and$
$          Elizabeth J. Watkins\inst{1}$
$          \and$
$          Brad C. Whitmore\inst{12}$
$          \and$
$          Ashley T. Barnes\inst{5,13}$
$          \and$
$          Enrico Congiu\inst{14}$
$          \and$
$          Daniel~A.~Dale\inst{15}$
$          \and$
$          Kathryn Grasha\inst{16}\thanks{ARC DECRA Fellow}$
$          \and$
$          Kirsten L. Larson\inst{17}$
$          \and$
$          Janice C. Lee\inst{18,19}$
$          \and$
$          J. Eduardo Méndez-Delgado\inst{1}$
$          \and$
$          David A. Thilker\inst{20}$
$          \and$
$          Thomas G. Williams\inst{21}$
$          }$
$   \institute{Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstr. 12-14, D-69120 Heidelberg, Germany$
$   \and$
$   Institüt  für Theoretische Astrophysik, Zentrum für Astronomie der Universität Heidelberg, Albert-Ueberle-Strasse 2, 69120 Heidelberg, Germany$
$   \and$
$   International Centre for Radio Astronomy Research, University of Western Australia, 7 Fairway, Crawley, 6009 WA, Australia$
$   \and$
$   INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157 Firenze, Italy$
$   \and$
$   European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany$
$   \and$
$    Univ Lyon, Univ Lyon1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval France$
$    \and$
$     Interdisziplinäres Zentrum für Wissenschaftliches Rechnen der Universität Heidelberg, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany$
$     \and$
$   Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA$
$    \and$
$    Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA$
$    \and$
$    Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium$
$    \and$
$    Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany$
$    \and$
$    Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD, 21218, USA$
$    \and$
$    Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany$
$    \and$
$    European Southern Observatory (ESO), Alonso de Córdova 3107, Casilla 19, Santiago 19001, Chile$
$    \and$
$    Department of Physics \& Astronomy, University of Wyoming, Laramie, WY 82071, USA$
$    \and$
$    Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia$
$    \and$
$    AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA$
$    \and$
$    Gemini Observatory/NSF’s NOIRLab, 950 N. Cherry Avenue, Tucson, AZ, USA$
$        \and$
$    Steward Observatory, University of Arizona, 933 N Cherry Ave, Tucson, AZ 85721, USA$
$    \and$
$    Department of Physics \& Astronomy, Bloomberg Center for Physics and Astronomy, Johns Hopkins University, 3400 N. Charles Street, Baltimore, MD 21218$
$    \and$
$    Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK$
$             }$
$   \date{Received ????? ??, 2023; accepted ????? ??, ????}$
$  \abstract{Stellar feedback is a key contributor to$
$   the morphology and dynamics of the interstellar medium in star-forming galaxies. In particular, energy and momentum input from massive stars can drive the turbulent motions in the gas, but the dominance and efficiency of this process are unclear. The study of ionised superbubbles enables quantitative constraints to be placed on the energetics of stellar feedback.}{We directly compare the kinetic energy of$
$   expanding superbubbles and the turbulent motions in the interstellar medium with the mechanical energy deposited by massive stars in the form of winds and supernovae. With such a comparison we aim to answer whether the stellar feedback is responsible for$
$   the observed turbulent motions and to quantify the fraction of mechanical energy retained in the superbubbles.}{We investigate the ionised gas morphology, excitation properties, and kinematics in 19 nearby star-forming galaxies from the PHANGS-MUSE survey. Based on the distribution of the flux and velocity dispersion in the \Ha line, we select \nregs regions of locally elevated velocity dispersion (\sigmaHa>45\kms), including at least \nshells expanding superbubbles. We analyse these regions and relate their properties to those of the young stellar associations and star clusters identified in PHANGS-HST data.}{We find a good correlation between the kinetic energy of the ionised gas and the total mechanical energy input from supernovae and stellar winds from the stellar associations. At the same time, the contribution of mechanical energy injected by the supernovae alone is not sufficient to explain the measured kinetic energy of the ionised gas, which implies that pre-supernova feedback in the form of radiation/thermal pressure and winds is necessary. We find that the gas kinetic energy decreases with metallicity for our sample covering Z=0.5-1.0   Z_\odot, reflecting the lower impact of stellar feedback. For the sample of superbubbles, we find that about 40\% of the young stellar associations are preferentially located in their rims.$
$   We also find a slightly higher (by \sim 15\%) fraction of the youngest (1--2.5~Myr) stellar associations in the rims of the superbubbles than in the centres, and the opposite trend for older associations, which implies possible propagation or triggering of star formation.}{Stellar feedback is the dominant source for powering the ionised gas in regions of locally (on 50--500~pc scale) elevated velocity dispersion, with a typical efficiency of 10-20\%. Accounting for pre-supernovae feedback is required to set up the energy balance between gas and stars. }$
$   \keywords{Galaxies: ISM --$
$                ISM: kinematics and dynamics --$
$                ISM: bubbles -- Galaxies: star formation$
$               }$
$   \maketitle$
$\n\end{document}\end{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand{\red}{\textcolor{red}}$
$\newcommand{\green}{\textcolor{green}}$</div>



<div id="title">

# Quantifying the energy balance between the turbulent ionised gas and young stars

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2307.10277-b31b1b.svg)](https://arxiv.org/abs/2307.10277)<mark>Appeared on: 2023-07-21</mark> -  _31 pages (including 5 pages in appendix), 19 figures, the abstract is abridged; submitted to A&A (in mid May; awaiting report)_

</div>
<div id="authors">

O. V. Egorov, et al. -- incl., <mark>K. Kreckel</mark>, <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** Stellar feedback is a key contributor to   the morphology and dynamics of the interstellar medium in star-forming galaxies. In particular, energy and momentum input from massive stars can drive the turbulent motions in the gas, but the dominance and efficiency of this process are unclear. The study of ionised superbubbles enables quantitative constraints to be placed on the energetics of stellar feedback. We directly compare the kinetic energy of   expanding superbubbles and the turbulent motions in the interstellar medium with the mechanical energy deposited by massive stars in the form of winds and supernovae. With such a comparison we aim to answer whether the stellar feedback is responsible for   the observed turbulent motions and to quantify the fraction of mechanical energy retained in the superbubbles. We investigate the ionised gas morphology, excitation properties, and kinematics in 19 nearby star-forming galaxies from the PHANGS-MUSE survey. Based on the distribution of the flux and velocity dispersion in the $\Ha$ line, we select $\nregs$ regions of locally elevated velocity dispersion ( $\sigmaHa$ $>45\kms$ ), including at least $\nshells$ expanding superbubbles. We analyse these regions and relate their properties to those of the young stellar associations and star clusters identified in PHANGS-HST data. We find a good correlation between the kinetic energy of the ionised gas and the total mechanical energy input from supernovae and stellar winds from the stellar associations. At the same time, the contribution of mechanical energy injected by the supernovae alone is not sufficient to explain the measured kinetic energy of the ionised gas, which implies that pre-supernova feedback in the form of radiation/thermal pressure and winds is necessary. We find that the gas kinetic energy decreases with metallicity for our sample covering $Z=0.5-1.0   Z_\odot$ , reflecting the lower impact of stellar feedback. For the sample of superbubbles, we find that about 40 \% of the young stellar associations are preferentially located in their rims.   We also find a slightly higher (by $\sim 15$ \% ) fraction of the youngest (1--2.5 Myr) stellar associations in the rims of the superbubbles than in the centres, and the opposite trend for older associations, which implies possible propagation or triggering of star formation. Stellar feedback is the dominant source for powering the ionised gas in regions of locally (on 50--500 pc scale) elevated velocity dispersion, with a typical efficiency of $10-20$ \% . Accounting for pre-supernovae feedback is required to set up the energy balance between gas and stars.

</div>

<div id="div_fig1">

<img src="tmp_2307.10277/./figs/NGC4254_isigma.png" alt="Fig9" width="100%"/>

**Figure 9. -** Localization of the regions of locally elevated $\Ha$ velocity dispersion (cyan ellipses) in NGC 4254 galaxy identified based on the `intensity -- velocity dispersion' ($\IS$) diagnostics overlaid on the $\Ha$ surface brightness map (left panel) and the classification map (central panel; see text). The $\IS$ diagram is shown on right panel. The black solid line shows the mean value of $\sigmaHa$ in the galaxy, and dashed grey lines show its 1$\sigma$ uncertainty.
    See Fig \ref{fig:isigma_all} for the rest of the PHANGS-MUSE galaxies. (*fig:isigma*)

</div>
<div id="div_fig2">

<img src="tmp_2307.10277/./figs/energy_ha_all_snw.png" alt="Fig18.1" width="25%"/><img src="tmp_2307.10277/./figs/energy_ha_all_sn.png" alt="Fig18.2" width="25%"/><img src="tmp_2307.10277/./figs/energy_ha_all_snw_s2dens.png" alt="Fig18.3" width="25%"/><img src="tmp_2307.10277/./figs/energy_ha_all_sn_s2dens.png" alt="Fig18.4" width="25%"/>

**Figure 18. -** Dependence of the kinetic energy of ionised gas $E_{kin}$ in the regions of locally elevated velocity dispersion on the total mechanical energy input from the stellar associations in the form of supernovae and stellar winds (left panels), and supernovae only (right panels). Top and bottom panels correspond to the $n_e^{(min)}$ and $n_e^{(max)}$ density measurements (thus representing upper and lower limits of $E_{kin}$, respectively). Median value of logarithmic errors is shown in bottom-right corner of each panel. All regions with errors 2 times larger than presented are excluded from the plots. Colour encodes the $\SII$Ha lines ratio tracing the dominant gas excitation mechanism ($\SII$Ha$>0.4$ is likely produced by shocks; left panels), or the relative contribution of the SNe to the total mechanical energy input according to the starburst99 models for the corresponding mass and age of each cluster. Blue, orange, green and red lines correspond to mechanical stellar feedback energy efficiency $\eta = 100, 30, 5, 1$\%, respectively. Spearman correlation coefficient $\rho$ is given above the plots. (*fig:energies*)

</div>
<div id="div_fig3">

<img src="tmp_2307.10277/./figs/SB99_energy.png" alt="Fig3" width="100%"/>

**Figure 3. -** Evolution of the cumulative mechanical energy input to the ISM ($E_{\rm mech}$) normalized by the mass of star cluster ($M_{\rm cls}$)  for different metallicity (traced by different colours) according to starburst99 ([Leitherer, Schaerer and Goldader 1999](), [Leitherer, Ekström and Meynet 2014]())  models. The contribution produced by stellar winds alone is shown by the solid line, while the dashed line corresponds to the impact of both stellar winds and supernovae. (*fig:sb99*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2307.10277"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

231  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
